In [ ]:
# 00 ) Packages
import pandas as pd
import numpy as np
import xlrd
import os
import pprint
from tkinter import filedialog as fd
os.chdir(r"C:\Users\pallist\Desktop\ΤΡΕΧΟΝΤΑ\testing folder")
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.options.display.max_rows = 999
pd.options.mode.chained_assignment = None 

In [ ]:
# 0 ) Importing Files
onomata = pd.read_excel("ΒΑΣΗ ΟΝΟΜΑΤΩΝ (ΑΡΣΕΝΙΚΑ).xlsx")
lista_onomaton = onomata.iloc[:,1]
file= fd.askopenfilename()
stoixeia = pd.read_excel(file)

In [ ]:
# 1) Insert columns of raw data into a new dataframe (if there is extra informtion about the person, add those columns too)

test = pd.DataFrame()
test["Ton"] = ""
test["Surname"] = stoixeia.iloc[:,0]
test["Name"] = stoixeia.iloc[:,1]
test["Father"] = stoixeia.iloc[:,2]

if len(stoixeia.columns) > 3 and not stoixeia.iloc[:,3].str.isnumeric().all():
    test["City"] = stoixeia.iloc[:,3]
    test["Address"] = stoixeia.iloc[:,4].astype(str)
    test["Number"] = stoixeia.iloc[:,5]
    test["Number"] = test["Number"].astype(str)
    if  is_numeric_dtype(stoixeia.iloc[:,6]):
        test["TK"] = stoixeia.iloc[:,6].fillna(0).astype(int).astype(str)
    else :
        test["TK"]  = ""
    if is_numeric_dtype(stoixeia.iloc[:,7]):
        test["ΑΦΜ"] = stoixeia.iloc[:,7].fillna(0).astype(int).astype(str)
    else :
        test["ΑΦΜ"]  = ""    
    test["stoixeia"] = ""
test.columns

In [ ]:
test

In [ ]:
# 2) Create hashmap dictionary για τις εξαιρέσεις των άρθρων και πόλεων 


ton_list = ["ΜΙΧΑΗΛ", "ΕΜΜΑΝΟΥΗΛ", "ΣΠΥΡΙΔΩΝ", "ΑΧΙΛΛΕΥΣ", "ΟΔΥΣΣΕΥΣ"]



keys = ["ΧΑΝΙΑ", "ΦΑΡΣΑΛΑ", "ΙΩΑΝΝΙΝΑ", "ΣΕΠΟΛΙΑ", "ΤΡΙΚΑΛΑ","ΑΝΩ ΛΙΟΣΙΑ", "ΚΑΛΥΒΙΑ", "ΜΕΓΑΡΑ","ΝΈΟ ΗΡΑΚΛΕΙΟ", "ΠΑΛΑΙΟ ΦΑΛΗΡΟ", "ΝΕΑ ΙΩΝΙΑ", "ΑΓΙΟΙ ΑΝΑΡΓΥΡΟΙ", "ΑΓΙΑ ΠΑΡΑΣΚΕΥΗ"]

values = ["ΧΑΝΙΩΝ", "ΦΑΡΣΑΛΩΝ", "ΙΩΑΝΝΙΝΩΝ", "ΣΕΠΟΛΙΩΝ","ΤΡΙΚΑΛΩΝ", "ΑΝΩ ΛΙΟΣΙΩΝ", "ΚΑΛΥΒΙΩΝ", "ΜΕΓΑΡΩΝ", "ΝΕΟΥ ΗΡΑΚΛΕΙΟΥ", "ΠΑΛΑΙΟΥ ΦΑΛΗΡΟΥ", "ΝΕΑΣ ΙΩΝΙΑΣ" , "ΑΓΙΩΝ ΑΝΑΡΓΥΡΩΝ", "ΑΓΙΑΣ ΠΑΡΑΣΚΕΥΗΣ"]

city_dict = dict(zip(keys, values))

In [ ]:
#  3) Ονόματα

def names () :
    ton_list = []
    for t in test["Name"] :
            x = str(t)
            for ton1 in lista_onomaton :
                if x == ton1:
                    ton_list.append("Τον")            
            if x[-2:] == "ΟΣ" or x[-2:] == "ΗΣ" :
                ton_list.append("Τον")
            elif x[-1:] == "Α" or x[-1:] == "Η":
                ton_list.append("Την")
            else :
                ton_list.append("Την")    
    test["Ton"] = ton_list

    surname_list = []
    
    
    for t in test["Surname"] :
        x = str(t)
        if x[-1] == "Σ" :
            x = x[:-1]
        surname_list.append(t)        
    test["Surname"] = surname_list        

    name_list = []

    for t in test["Name"] :
        x = str(t)
        if x[-1] == "Σ" :
            x = t[:-1]
        name_list.append(t)

    test["Name"] = name_list

    father_list = []

    for t in test["Father"] :
        x = str(t)
        if x[-2:] == "ΗΣ" or x[-2:] == "ΑΣ" :
            x = x[:-1]
        elif x[-2:] == "ΟΣ":
            x = x[:-1] + "Y"
    #     print(x)
        father_list.append(x)        

    test["Father"] = father_list

    test


In [ ]:
# 4) Πόλεις

def cities() :    
    
    city_list = []

    for t in test["City"] :
        x = str(t)

        for city1, city2 in city_dict.items() :
            if x == city1:
                x = city2
        
        if x[-1] == "Α" or x[-1] == "Η" :
            x = x + "Σ"
        
        elif x[-2:] == "ΑΣ" :
            x = x[:-1]    
    
        elif x[-2:] == "ΕΣ" or x[-2:] == "ΟΙ":
            x = x[:-2] + "ΩΝ"    
            
        elif x[-1] == "Ι"  :
            x = x + "ΟΥ"

        elif x[-1] == "Ο"  :
            x = x + "Υ"

        elif x[-2:] == "ΟΣ" :
            x = x[:-1] + "Y"

        city_list.append(x)

    test["City"] = city_list

In [ ]:
#   5) Λογαριασμός

def account() :
    stoixeia['Account Date'] = pd.to_datetime(stoixeia['ΗΜΕΡ.ΤΕΛΙΚΟΥ ΛΟΓΙΑΡΙΑΣΜΟΥ']).dt.strftime('%d-%m-%Y')
    test['Νumber/date account'] = stoixeia['ΑΡ.ΤΕΛΙΚΟΣ ΛΟΓΑΡΙΑΣΜΟΥ'].astype('string') + "/" + stoixeia['Account Date'].astype('string')
   




In [ ]:
# 6) Αίτηση - Define filtered dataframes  

cols = ['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ', 'ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']

loc_date_full = stoixeia.loc[stoixeia[['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ']].notnull().all(1)]
date_full = stoixeia[['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ']].notnull().all(1)

loc_number_full = stoixeia.loc[stoixeia[['ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']].notnull().all(1)]
number_full = stoixeia[['ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']].notnull().all(1)

loc_date_na = stoixeia.loc[stoixeia[['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ']].isnull().all(1)]
date_na = stoixeia[['ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ']].isnull().all(1)

loc_number_na = stoixeia.loc[stoixeia[['ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']].isnull().all(1)]
number_na = stoixeia[['ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ']].isnull().all(1)   


# Μόνο ημερομηνία

date_full_number_na = stoixeia.loc[(date_na)&(number_na)]

# Μόνο αριθμός λογαριασμού

number_full_date_na = stoixeia.loc[(number_full)&(date_na)]

# Γεμάτα
stoixeia[stoixeia[cols].notnull().all(1)]

# Κενά
stoixeia[stoixeia[cols].isna().all(1)] # Find both nulls columns application

In [60]:
# 8) Split παρόχους και παροχές

dict_paroxoi = {} # 1) Για κάθε ξεχωριστό πάροχο δημιούργησε το φιλτραρισμένο df και εισήγαγέ το στο λεξικό παρόχων
count_paroxos = {}
filtered_df_dict = {}

for paroxos in stoixeia['ΣΥΜΒΑΣΗ '].unique() : # 2) Για κάθε df παρόχου στο λεξικο παρόχων,δημιούργησε στήλη Pool με flag το count της τιμής σε κάθε σειρά στην στήλη ΑΦΜ 
    dict_paroxoi[paroxos] = stoixeia[stoixeia.iloc[:,18]  == paroxos]  # Δημιούργησε dataframe για κάθε πάροχο
    count_paroxos[paroxos] = sorted(list(dict_paroxoi[paroxos][['ΑΦΜ ']].value_counts().unique())) # 3) Για κάθε df παρόχου δημιούργησε λίστες με τον αριθμό countif      
    dict_paroxoi[paroxos]['Pool'] = dict_paroxoi[paroxos]['ΑΦΜ '].map(dict_paroxoi[paroxos]['ΑΦΜ '].value_counts())

voldf = dict_paroxoi['VOLTON']
voldf

# 4) Δημιουργία dataframe volton_count1, volton_count2, volton_count3  etc ...

   
for paroxos_name,paroxos_df in dict_paroxoi.items() :  # 1st step) Iterate over every paroxos df
   for paroxos_name_count, count_flag in count_paroxos.items(): # 2nd step) Iterate over λεξικό με λίστα count για κάθε πάροχο
    if paroxos_df.iloc[:,18] == paroxos_name_count : # 3d step) Get only the relative df for each paroxo in count_paroxos
                                                    # 4th step) Filter by each item iin the λίστα count (values of count_paroxos λεξικό)











    # φiltered_df_dict[Pool] = dict_paroxoi[paroxos][dict_paroxoi[paroxos][['Pool']]==count_paroxos[counted]] #4) Για κάθε πάροοχο_df, itereate over αριθμό countif στην αντίστοιχη λίστα μοναδικών αριθμών πλήθους παροχών, δημιούργησε το αντίστοιχο filtered df και σώσε το στο κεντρικό filtered_dict 

0      VOLTON
1      VOLTON
2      VOLTON
3      VOLTON
4      VOLTON
5      VOLTON
6      VOLTON
7      VOLTON
8      VOLTON
9      VOLTON
10     VOLTON
11     VOLTON
12     VOLTON
13     VOLTON
14     VOLTON
15     VOLTON
16     VOLTON
17     VOLTON
18     VOLTON
19     VOLTON
20     VOLTON
21     VOLTON
22     VOLTON
23     VOLTON
24     VOLTON
25     VOLTON
26     VOLTON
27     VOLTON
28     VOLTON
29     VOLTON
30     VOLTON
31     VOLTON
32     VOLTON
33     VOLTON
34     VOLTON
35     VOLTON
38     VOLTON
40     VOLTON
41     VOLTON
42     VOLTON
43     VOLTON
44     VOLTON
45     VOLTON
46     VOLTON
47     VOLTON
48     VOLTON
49     VOLTON
50     VOLTON
51     VOLTON
52     VOLTON
53     VOLTON
54     VOLTON
55     VOLTON
56     VOLTON
57     VOLTON
58     VOLTON
59     VOLTON
60     VOLTON
61     VOLTON
62     VOLTON
63     VOLTON
64     VOLTON
65     VOLTON
66     VOLTON
67     VOLTON
68     VOLTON
69     VOLTON
70     VOLTON
71     VOLTON
72     VOLTON
73     VOLTON
74    

In [ ]:
# Final) Create the dataframe
# # names()
if {'ΗΜΕΡ.ΥΠΟΓΡ.ΑΙΤΗΣΗΣ', 'ΑΡΙΘΜΟΣ ΑΙΤΗΣΗΣ','ΑΡ.ΤΕΛΙΚΟΣ ΛΟΓΑΡΙΑΣΜΟΥ','ΗΜΕΡ.ΤΕΛΙΚΟΥ ΛΟΓΙΑΡΙΑΣΜΟΥ'}.issubset(stoixeia.columns):
    print("Συγχαρητήρια είναι ένα υγιέστατο εξώδικο!")
    account()
    
elif len(stoixeia.columns) > 4 :
    cities()
    test["stoixeia_full"] = test["Ton"] + " " + test["Surname"] + " " + test["Name"] + " " + "του " + test['Father'] + ", κάτοικο "+ test["City"] + ", οδός " + test["Address"] + " αρ. " + test["Number"] + ", ΤΚ " + test["TK"] + ", με ΑΦΜ " + test["ΑΦΜ"] + "."



test["stoixeia"]=  test["Ton"] + " " + test["Surname"] + " " + test["Name"] 
# filename = file[60:80]
# test.to_excel(f"{filename}.xlsx", index = False)

# os.startfile(f"{filename}.xslx")


In [ ]:
# Παράδειγμα για split
# 5) κάνε το split by (pool) rows
(par1)(z)cut =  pool(z).iloc[::(z), :] #1 paroxi Volton_pool_
(par1)(x)cut =  pool(x).iloc[::(x), :] #2 paroxi Volton_pool_
(par1)(y)_cut = pool(y).iloc[::(y), :]   #3 paroxi Volton_pool_


(par2)(z)cut =  pool(z).iloc[::(z), :] #1 paroxi NOVA pool
(par2)(x)cut =  pool(x).iloc[::(x), :] #2 paroxi NOVA pool
(par2)(y)cut = pool(y).iloc[::(y), :]   #3 paroxi NOVA pool_
